In [1]:
import random as rdm
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("salario.csv")
df.head()

,id,pais,estado,cidade,genero,data,salario
0,1,United States,Alabama,Mobile,Male,10/01/1987,$5407.86
1,2,Portugal,Porto,Rego,Male,29/11/2007,$7833.82
2,3,Sweden,Kalmar,Emmaboda,Male,23/06/1984,$5530.85
3,4,Sweden,Västmanland,Sala,Female,04/11/1983,$2803.76
4,5,France,Bretagne,Quimperlé,Female,18/11/1990,$2201.87


In [3]:
df.describe()

,id
count,1000.000000
mean,500.500000
std,288.819436
min,1.000000
25%,250.750000
50%,500.500000
75%,750.250000
max,1000.000000


## Unindo os atributos de localidade

In [4]:
ids = df.id
cities = df.cidade
states = df.estado
countries = df.pais

In [5]:
locality = []
for i in range(len(cities)):
    locality.append([ids[i], str('{}, {}, {}'.format(cities[i], states[i], countries[i]))])

In [6]:
df_locality = pd.DataFrame(locality, columns=['id', 'localidade'])
df_locality.head()

,id,localidade
0,1,"Mobile, Alabama, United States"
1,2,"Rego, Porto, Portugal"
2,3,"Emmaboda, Kalmar, Sweden"
3,4,"Sala, Västmanland, Sweden"
4,5,"Quimperlé, Bretagne, France"


## Unindo os dataframes

In [7]:
join_dataset = pd.concat([df, df_locality], axis=1, join='inner')
join_dataset.head()

,id,pais,estado,cidade,genero,data,salario,id,localidade
0,1,United States,Alabama,Mobile,Male,10/01/1987,$5407.86,1,"Mobile, Alabama, United States"
1,2,Portugal,Porto,Rego,Male,29/11/2007,$7833.82,2,"Rego, Porto, Portugal"
2,3,Sweden,Kalmar,Emmaboda,Male,23/06/1984,$5530.85,3,"Emmaboda, Kalmar, Sweden"
3,4,Sweden,Västmanland,Sala,Female,04/11/1983,$2803.76,4,"Sala, Västmanland, Sweden"
4,5,France,Bretagne,Quimperlé,Female,18/11/1990,$2201.87,5,"Quimperlé, Bretagne, France"


## Removendo atributos

In [29]:
dataset = join_dataset.drop(columns = ['id', 'pais', 'estado', 'cidade', 'salario'])
dataset.head()

,genero,data,localidade
0,Male,10/01/1987,"Mobile, Alabama, United States"
1,Male,29/11/2007,"Rego, Porto, Portugal"
2,Male,23/06/1984,"Emmaboda, Kalmar, Sweden"
3,Female,04/11/1983,"Sala, Västmanland, Sweden"
4,Female,18/11/1990,"Quimperlé, Bretagne, France"


## Funções de generalização

In [9]:
def date_generalization(dataset, level):
    new_dataset = dataset.copy()
    if level == 1:
        # date '**/MM/YYYY'
        for data in range(len(new_dataset)):
            new_dataset.iloc[data]['data'] = str('**' + new_dataset.iloc[data]['data'][2:10])
    elif level == 2:
        # date '**/**/YYYY'
        for data in range(len(new_dataset)):
            new_dataset.iloc[data]['data'] = str('**/**' + new_dataset.iloc[data]['data'][5:10])
    elif level == 3:
        # date'**/**/****'
        for data in range(len(new_dataset)):
            new_dataset.iloc[data]['data'] = str('**/**/****')
        
    return new_dataset

In [10]:
def locality_generalization(dataset, level):
    new_dataset = dataset.copy()
    if level == 0:
        # locality 'city'
        for data in range(len(new_dataset)):
            new_dataset.iloc[data]['localidade'] = new_dataset.iloc[data]['localidade'].split(',')[0]
    elif level == 1:
        # locality 'state,'
        for data in range(len(new_dataset)):
            new_dataset.iloc[data]['localidade'] = new_dataset.iloc[data]['localidade'].split(', ')[1]
    elif level == 2:
        # locality 'country'
        for data in range(len(new_dataset)):
            new_dataset.iloc[data]['localidade'] = new_dataset.iloc[data]['localidade'].split(', ')[2]
    elif level == 3:
        # locality '*'
        for data in range(len(new_dataset)):
            new_dataset.iloc[data]['localidade'] = '*'
        
    return new_dataset

In [11]:
def gender_generalization(dataset, level):
    new_dataset = dataset.copy()
    if level == 1:
        # gender '*'
        for data in range(len(new_dataset)):
            new_dataset.iloc[data]['genero'] = '*'
            
    return new_dataset

## Gerando as Classes de Equivalência

In [90]:
dataset_date_gen = date_generalization(dataset, 0)
dataset_gender_gen = gender_generalization(dataset_date_gen, 0)
dataset_loc_gen = locality_generalization(dataset_gender_gen, 2)

dataset_loc_gen.head()

,genero,data,localidade
0,Male,10/01/1987,United States
1,Male,29/11/2007,Portugal
2,Male,23/06/1984,Sweden
3,Female,04/11/1983,Sweden
4,Female,18/11/1990,France


In [60]:
# Teste baseado na distância entre os atributos para definir as classes de equivalência

dataset_date_gen = date_generalization(dataset, 1)
dataset_gender_gen = gender_generalization(dataset_date_gen, 0)
dataset_loc_gen = locality_generalization(dataset_gender_gen, 1)
groups = generate_group(dataset_loc_gen)
sort_dict(groups)

i = 0
k = 2
new_group = list()
for data in groups:
    if groups[data] < k:
        i += 1
i

14

In [122]:
def define_groups(dataset_dict, k):
    print('##############################################################')
    print(dataset_dict)
    flag = False
    for value, level in dataset_dict.values():
        if value < k:
            flag = True
    
    if not(flag):
        return dataset_dict
    
    without_group = dict()
    with_group = dict()
    
    for data in dataset_dict:
        if dataset_dict[data][0] < k:
            without_group[data] = dataset_dict[data]
        else:
            with_group[data] = dataset_dict[data]
        
    with_group.update(generate_equivalent_classes(without_group, k)) # concatenate dictionaries
    return define_groups(with_group, k)

In [117]:
def generate_equivalent_classes(dataset_dict, k):
    locality_level = list()
    date_level = list()
    gender_level = list()
    
    for value, level in dataset_dict.values():
            locality_level.append(level[0])
            date_level.append(level[1])
            gender_level.append(level[2])
    
    avgs = [np.average(locality_level), np.average(locality_level), np.average(locality_level)]
    attr = 0
    i = 0
    min_avg = 0
    
    for avg in avgs:
        if avg < min_avg:
            min_avg = avg
            attr = i
        i += 1

    new_dataset = pd.DataFrame(list(dataset_dict.keys()), columns=['localidade', 'data', 'genero'])
    level = [np.floor(avgs[0]), np.floor(avgs[1]), np.floor(avgs[2])]
    if attr == 0:    
        new_group = locality_generalization(new_dataset, level[0] + 1)
    elif attr == 1:
        new_group = date_generalization(new_dataset, 1, level[1] + 1)
    elif attr == 2:
        new_group = gender_generalization(new_dataset, 0, level[2] + 1)
    
    
    return generate_group(new_group, level)

## Estudo sobre os dados

In [118]:
def sort_dict(unordered_dict):
    sorted_dict = dict()
    for key in unordered_dict:
        sorted_dict[key] = unordered_dict[key]

    return sorted(sorted_dict.items(), key=lambda kv: kv[1][0])

In [119]:
def generate_group(dataset, levels):
    # levels = [locality, date, gender]
    group_dict = dict()

    for data in range(len(dataset)):
        register = dataset.iloc[data]
        if (register['localidade'], register['data'], register['genero']) in group_dict.keys():
            group_dict[(register['localidade'], register['data'], register['genero'])][0] += 1
        else:
            group_dict[(register['localidade'], register['data'], register['genero'])] = [1, levels]
    return group_dict

In [123]:
groups = generate_group(dataset_loc_gen, [2, 0, 0])
sort_dict(groups)

[(('United States', '10/01/1987', 'Male'), [1, [2, 0, 0]]),
 (('Portugal', '29/11/2007', 'Male'), [1, [2, 0, 0]]),
 (('Sweden', '23/06/1984', 'Male'), [1, [2, 0, 0]]),
 (('Sweden', '04/11/1983', 'Female'), [1, [2, 0, 0]]),
 (('France', '18/11/1990', 'Female'), [1, [2, 0, 0]]),
 (('United States', '10/07/1981', 'Female'), [1, [2, 0, 0]]),
 (('France', '23/03/1989', 'Female'), [1, [2, 0, 0]]),
 (('Mexico', '02/05/1990', 'Male'), [1, [2, 0, 0]]),
 (('United States', '27/11/1992', 'Male'), [1, [2, 0, 0]]),
 (('France', '19/01/2007', 'Female'), [1, [2, 0, 0]]),
 (('United States', '29/01/2007', 'Male'), [1, [2, 0, 0]]),
 (('Sweden', '25/03/1995', 'Male'), [1, [2, 0, 0]]),
 (('Sweden', '15/01/1996', 'Female'), [1, [2, 0, 0]]),
 (('Sweden', '31/10/2002', 'Female'), [1, [2, 0, 0]]),
 (('United States', '14/11/1980', 'Female'), [1, [2, 0, 0]]),
 (('Sweden', '05/05/1985', 'Female'), [1, [2, 0, 0]]),
 (('Canada', '30/04/2009', 'Male'), [1, [2, 0, 0]]),
 (('United States', '06/05/1983', 'Female'),

In [124]:
define_groups(groups, 2)

##############################################################
{('United States', '10/01/1987', 'Male'): [1, [2, 0, 0]], ('Portugal', '29/11/2007', 'Male'): [1, [2, 0, 0]], ('Sweden', '23/06/1984', 'Male'): [1, [2, 0, 0]], ('Sweden', '04/11/1983', 'Female'): [1, [2, 0, 0]], ('France', '18/11/1990', 'Female'): [1, [2, 0, 0]], ('United States', '10/07/1981', 'Female'): [1, [2, 0, 0]], ('France', '23/03/1989', 'Female'): [1, [2, 0, 0]], ('Mexico', '02/05/1990', 'Male'): [1, [2, 0, 0]], ('United States', '27/11/1992', 'Male'): [1, [2, 0, 0]], ('France', '19/01/2007', 'Female'): [1, [2, 0, 0]], ('United States', '29/01/2007', 'Male'): [1, [2, 0, 0]], ('Sweden', '25/03/1995', 'Male'): [1, [2, 0, 0]], ('Sweden', '15/01/1996', 'Female'): [1, [2, 0, 0]], ('Sweden', '31/10/2002', 'Female'): [1, [2, 0, 0]], ('United States', '14/11/1980', 'Female'): [1, [2, 0, 0]], ('Sweden', '05/05/1985', 'Female'): [1, [2, 0, 0]], ('Canada', '30/04/2009', 'Male'): [1, [2, 0, 0]], ('United States', '06/05/1983',

##############################################################
{('France', '15/11/2005', 'Female'): [2, [2, 0, 0]], ('Portugal', '29/11/1987', 'Male'): [2, [2, 0, 0]], ('France', '06/06/2002', 'Female'): [2, [2, 0, 0]], ('*', '10/01/1987', 'Male'): [1, [2.0, 2.0, 2.0]], ('*', '29/11/2007', 'Male'): [1, [2.0, 2.0, 2.0]], ('*', '23/06/1984', 'Male'): [1, [2.0, 2.0, 2.0]], ('*', '04/11/1983', 'Female'): [1, [2.0, 2.0, 2.0]], ('*', '18/11/1990', 'Female'): [1, [2.0, 2.0, 2.0]], ('*', '10/07/1981', 'Female'): [1, [2.0, 2.0, 2.0]], ('*', '23/03/1989', 'Female'): [1, [2.0, 2.0, 2.0]], ('*', '02/05/1990', 'Male'): [1, [2.0, 2.0, 2.0]], ('*', '27/11/1992', 'Male'): [1, [2.0, 2.0, 2.0]], ('*', '19/01/2007', 'Female'): [1, [2.0, 2.0, 2.0]], ('*', '29/01/2007', 'Male'): [1, [2.0, 2.0, 2.0]], ('*', '25/03/1995', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '15/01/1996', 'Female'): [1, [2.0, 2.0, 2.0]], ('*', '31/10/2002', 'Female'): [1, [2.0, 2.0, 2.0]], ('*', '14/11/1980', 'Female'): [1, [2.0, 2.0, 2.0]],

##############################################################
{('France', '15/11/2005', 'Female'): [2, [2, 0, 0]], ('Portugal', '29/11/1987', 'Male'): [2, [2, 0, 0]], ('France', '06/06/2002', 'Female'): [2, [2, 0, 0]], ('*', '25/03/1995', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '06/05/1983', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '28/02/1996', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '19/10/1981', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '26/10/1993', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '05/05/2007', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '25/03/1994', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '10/05/1997', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '26/01/2004', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '17/01/1981', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '16/06/1996', 'Female'): [3, [2.0, 2.0, 2.0]], ('*', '09/10/1994', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '27/11/2000', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '11/05/1994', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '14/10/2000', 'Female'): [2, [2.0, 2.0, 2.0]

##############################################################
{('France', '15/11/2005', 'Female'): [2, [2, 0, 0]], ('Portugal', '29/11/1987', 'Male'): [2, [2, 0, 0]], ('France', '06/06/2002', 'Female'): [2, [2, 0, 0]], ('*', '25/03/1995', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '06/05/1983', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '28/02/1996', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '19/10/1981', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '26/10/1993', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '05/05/2007', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '25/03/1994', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '10/05/1997', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '26/01/2004', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '17/01/1981', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '16/06/1996', 'Female'): [3, [2.0, 2.0, 2.0]], ('*', '09/10/1994', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '27/11/2000', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '11/05/1994', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '14/10/2000', 'Female'): [2, [2.0, 2.0, 2.0]

##############################################################
{('France', '15/11/2005', 'Female'): [2, [2, 0, 0]], ('Portugal', '29/11/1987', 'Male'): [2, [2, 0, 0]], ('France', '06/06/2002', 'Female'): [2, [2, 0, 0]], ('*', '25/03/1995', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '06/05/1983', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '28/02/1996', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '19/10/1981', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '26/10/1993', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '05/05/2007', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '25/03/1994', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '10/05/1997', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '26/01/2004', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '17/01/1981', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '16/06/1996', 'Female'): [3, [2.0, 2.0, 2.0]], ('*', '09/10/1994', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '27/11/2000', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '11/05/1994', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '14/10/2000', 'Female'): [2, [2.0, 2.0, 2.0]

...

##############################################################
{('France', '15/11/2005', 'Female'): [2, [2, 0, 0]], ('Portugal', '29/11/1987', 'Male'): [2, [2, 0, 0]], ('France', '06/06/2002', 'Female'): [2, [2, 0, 0]], ('*', '25/03/1995', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '06/05/1983', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '28/02/1996', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '19/10/1981', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '26/10/1993', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '05/05/2007', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '25/03/1994', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '10/05/1997', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '26/01/2004', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '17/01/1981', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '16/06/1996', 'Female'): [3, [2.0, 2.0, 2.0]], ('*', '09/10/1994', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '27/11/2000', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '11/05/1994', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '14/10/2000', 'Female'): [2, [2.0, 2.0, 2.0]

##############################################################
{('France', '15/11/2005', 'Female'): [2, [2, 0, 0]], ('Portugal', '29/11/1987', 'Male'): [2, [2, 0, 0]], ('France', '06/06/2002', 'Female'): [2, [2, 0, 0]], ('*', '25/03/1995', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '06/05/1983', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '28/02/1996', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '19/10/1981', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '26/10/1993', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '05/05/2007', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '25/03/1994', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '10/05/1997', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '26/01/2004', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '17/01/1981', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '16/06/1996', 'Female'): [3, [2.0, 2.0, 2.0]], ('*', '09/10/1994', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '27/11/2000', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '11/05/1994', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '14/10/2000', 'Female'): [2, [2.0, 2.0, 2.0]

##############################################################
{('France', '15/11/2005', 'Female'): [2, [2, 0, 0]], ('Portugal', '29/11/1987', 'Male'): [2, [2, 0, 0]], ('France', '06/06/2002', 'Female'): [2, [2, 0, 0]], ('*', '25/03/1995', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '06/05/1983', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '28/02/1996', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '19/10/1981', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '26/10/1993', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '05/05/2007', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '25/03/1994', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '10/05/1997', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '26/01/2004', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '17/01/1981', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '16/06/1996', 'Female'): [3, [2.0, 2.0, 2.0]], ('*', '09/10/1994', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '27/11/2000', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '11/05/1994', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '14/10/2000', 'Female'): [2, [2.0, 2.0, 2.0]

##############################################################
{('France', '15/11/2005', 'Female'): [2, [2, 0, 0]], ('Portugal', '29/11/1987', 'Male'): [2, [2, 0, 0]], ('France', '06/06/2002', 'Female'): [2, [2, 0, 0]], ('*', '25/03/1995', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '06/05/1983', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '28/02/1996', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '19/10/1981', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '26/10/1993', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '05/05/2007', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '25/03/1994', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '10/05/1997', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '26/01/2004', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '17/01/1981', 'Male'): [2, [2.0, 2.0, 2.0]], ('*', '16/06/1996', 'Female'): [3, [2.0, 2.0, 2.0]], ('*', '09/10/1994', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '27/11/2000', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '11/05/1994', 'Female'): [2, [2.0, 2.0, 2.0]], ('*', '14/10/2000', 'Female'): [2, [2.0, 2.0, 2.0]

KeyboardInterrupt: 

## Função de Precisão

$precision (D, Na, h, HGVA) = 1 - \frac{\sum_{i=1}^{Na} \sum_{j=1}^{|D|} \frac{h}{HGVA_i}}{|D| * Na}
$

- D: conjunto de dados
- Na: número de atributos semi-identificadores
- h: altura da hierarquia de generalização de valor do atributo Ai após anonimização
- |HGVAi|: altura máxima da hierarquia

In [46]:
def precision(D, Na, h, HGVA):
    summation = 0
    for i in range(Na):
        for j in range(len(D)):
            summation += (h[i] / (HGVA[i]))
    
    return 1 - (summation / (len(D) * Na))
#     return 1 - (summation / ((len(D) * Na)
            

In [47]:
precision(groups, 3, [2, 4, 0], [3, 4, 1])

0.4444444444444444